# WaveRoll - Jupyter Notebook Demo
> JavaScript Library for Comparative MIDI Piano-Roll Visualization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crescent-stdio/wave-roll/blob/main/docs/examples/jupyter-notebook/wave-roll-demo.ipynb) [![GitHub](https://img.shields.io/badge/GitHub-View_on_GitHub-blue?logo=github)](https://github.com/crescent-stdio/wave-roll)

This notebook demonstrates how to embed WaveRoll in Jupyter notebooks for interactive MIDI visualization and comparison.


## 1. Usage

### 1.1. Basic IFrame Embed

Embed the WaveRoll standalone demo using an IFrame:

**Note**: The standalone interface allows you to drag and drop your own MIDI and audio files for visualization and comparison.

In [ ]:
from IPython.display import IFrame
IFrame(src='https://crescent-stdio.github.io/wave-roll/standalone.html', width='100%', height='800px' )

### 1.2. IFrame with Overflow Auto

If you need scrollbar control on the iframe itself:


In [ ]:
from IPython.display import HTML

HTML("""
<iframe 
  src="https://crescent-stdio.github.io/wave-roll/standalone.html" 
  width="100%" 
  height="800px"
  style="overflow: auto; border: none;"
></iframe>
""")

## 2. Alternative: Using the Web Component Directly

You can also use the `<wave-roll>` web component directly in Jupyter notebooks with specific file paths:


In [ ]:
from IPython.display import HTML

html_content = """
<script type="module">
  import 'https://cdn.jsdelivr.net/npm/wave-roll@latest/dist/wave-roll.es.js';
</script>

<div style="width: 100%; height: 700px; overflow: auto;">
  <wave-roll
    style="width: 100%; height: 100%;"
    files='[
      {"path": "https://crescent-stdio.github.io/wave-roll/sample_midi/chopin_audio.mp3", "name": "Audio", "type": "audio"},
      {"path": "https://crescent-stdio.github.io/wave-roll/sample_midi/chopin_ground_truth.midi", "name": "Ground Truth", "type": "midi"},
      {"path": "https://crescent-stdio.github.io/wave-roll/sample_midi/chopin_model_b.midi", "name": "Model B", "type": "midi"},
      {"path": "https://crescent-stdio.github.io/wave-roll/sample_midi/chopin_model_y.midi", "name": "Model Y", "type": "midi"}
    ]'
  ></wave-roll>
</div>
"""

HTML(html_content)


### 2.1. Using Your Own Files

To use your own MIDI/audio files, you can:

1. Upload files to a publicly accessible URL (e.g., GitHub, cloud storage)
2. Use the file URLs in the `files` attribute
3. Or use empty files array `[]` and let users drag-and-drop files directly in the UI

Example with custom files:


In [ ]:
from IPython.display import HTML

# Empty files array - allows drag-and-drop file upload
html_content = """
<script type="module">
  import 'https://cdn.jsdelivr.net/npm/wave-roll@latest/dist/wave-roll.es.js';
</script>

<div style="width: 100%; height: 700px; overflow: auto;">
  <wave-roll
    style="width: 100%; height: 100%;"
    files='[]'
  ></wave-roll>
</div>
"""

HTML(html_content)


### 2.2. Dynamic File Configuration with Python

You can also configure files programmatically using Python variables:


In [ ]:
from IPython.display import HTML
import json

# Define your files in Python
files = [
    {
        "path": "https://your-domain.com/audio/track.mp3",
        "name": "Original Audio",
        "type": "audio"
    },
    {
        "path": "https://your-domain.com/midi/ground_truth.mid",
        "name": "Ground Truth",
        "type": "midi"
    },
    {
        "path": "https://your-domain.com/midi/model_output.mid",
        "name": "Model Output",
        "type": "midi"
    }
]

# Convert to JSON string
files_json = json.dumps(files)

# Create HTML with embedded files
html_content = f"""
<script type="module">
  import 'https://cdn.jsdelivr.net/npm/wave-roll@latest/dist/wave-roll.es.js';
</script>

<div style="width: 100%; height: 700px; overflow: auto;">
  <wave-roll
    style="width: 100%; height: 100%;"
    files='{files_json}'
  ></wave-roll>
</div>
"""

HTML(html_content)


# 3. Dynamic File Configuration with Google Colab

The Colab environment has difficulty accessing files using just the file paths from the Colab filesystem (`/content/...`). So we need to encode the files to base64 and use the Data URIs.

The following code:
1. Scans the `/content/sample_midi/` directory (You can change this to any directory containing your MIDI and audio files) for audio (`.mp3`, `.wav`, `.ogg`) and MIDI (`.midi`, `.mid`) files
2. Encodes each file to base64 and converts it to a Data URI with the appropriate MIME type
3. Builds a file list with name, type, and encoded content
4. Renders the WaveRoll component with the embedded files for interactive visualization

In [ ]:
from IPython.display import HTML, display
import json
import os
import base64

# Directory containing sample files
SAMPLE_MIDI_DIR = "/content/sample_midi/"

files_data = []

# Function to encode file to base64
def encode_file_to_base64(file_path):
    if os.path.exists(file_path):
        try:
            with open(file_path, "rb") as f:
                file_content = f.read()
                encoded_content = base64.b64encode(file_content).decode('utf-8')

                # Determine MIME type based on file extension
                file_extension = os.path.splitext(file_path)[1].lower()
                mime_type = "application/octet-stream" # Default
                if file_extension in ['.mp3']:
                    mime_type = "audio/mp3"
                elif file_extension in ['.wav']:
                    mime_type = "audio/wav"
                elif file_extension in ['.ogg']:
                    mime_type = "audio/ogg"
                elif file_extension in ['.midi', '.mid']:
                    mime_type = "audio/midi" # Standard MIME type for MIDI

                return f"data:{mime_type};base64,{encoded_content}"
        except Exception as e:
            print(f"Error encoding file {file_path}: {e}")
            return None
    return None

# Find and process all relevant files in the sample directory
if os.path.isdir(SAMPLE_MIDI_DIR):
    for file_name in os.listdir(SAMPLE_MIDI_DIR):
        file_path = os.path.join(SAMPLE_MIDI_DIR, file_name)
        if os.path.isfile(file_path):
            file_extension = os.path.splitext(file_name)[1].lower()
            file_type = None

            # Determine file type for wave-roll based on extension
            if file_extension in ['.mp3', '.wav', '.ogg']:
                file_type = 'audio'
            elif file_extension in ['.midi', '.mid']:
                file_type = 'midi'

            if file_type:
                data_uri = encode_file_to_base64(file_path)
                if data_uri:
                    files_data.append({
                        "path": data_uri,
                        "name": file_name,
                        "type": file_type
                    })
                else:
                    print(f"Could not encode file: {file_name}")
else:
    print(f"Error: Directory not found at {SAMPLE_MIDI_DIR}")


# Only proceed if files were successfully encoded
if files_data:
    # Convert to JSON string
    files_json = json.dumps(files_data)

    # Create HTML with embedded files using Data URIs
    html_content = f"""
    <script type="module">
      import 'https://cdn.jsdelivr.net/npm/wave-roll@latest/dist/wave-roll.es.js';
    </script>

    <div style="width: 100%; height: 800px; overflow: auto;">
      <wave-roll
        style="width: 100%; height: 100%;"
        files='{files_json}'
      ></wave-roll>
    </div>
    """

    display(HTML(html_content))
else:
    print("No supported audio or MIDI files found or successfully encoded in the sample directory.")